In [ ]:
#imports

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

In [ ]:
#load event

data_path = '../Data/train_100_events/'
event_prefix = 'event000001000'
hits, cells, particles, truth = load_event(os.path.join(data_path, event_prefix))
hits.head()

In [ ]:
#transorm spherical coordinates
import math as m

def spherical(x,y,z):
    a = x**2 + y**2
    r = m.sqrt(a + z**2)
    theta = m.atan2(z,m.sqrt(a))
    phi = m.atan2(y,x)
    return r, theta, phi

print(spherical(1, 1, 1))

In [ ]:
#prepare spherical coordinates

x, y = [], []

for index, row in hits.iterrows():
    r, theta, phi = spherical(row['x'], row['y'], row['z'])
    x.append(theta)
    y.append(phi)

In [ ]:
print(len(x), len(y))

In [ ]:
#plot
import seaborn as sns
plt.rcParams["figure.figsize"] = [20,20]

plt.scatter(x, y, s=1)
plt.show()

In [ ]:
#DBSCAN Clusterer

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

def predict(X, eps=0.008):
    cl = DBSCAN(eps, min_samples=1, algorithm='kd_tree')
    labels = cl.fit_predict(StandardScaler().fit_transform(X))
    return labels

In [ ]:
labels = predict(np.transpose([x,y]))

In [ ]:
print(labels)

In [ ]:
#score

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

submission = create_one_event_submission(0, hits, labels)
score = score_event(truth, submission)

print("Your score: ", score)

In [ ]:
def try_clustering(hits, X):
    labels = predict(X)
    submission = create_one_event_submission(0, hits, labels)
    score = score_event(truth, submission)
    print("Your score: ", score)
    

In [ ]:
#strange coordinate transform

def strange(x,y,z):
    r = np.sqrt(x**2 + y**2 + z**2)
    x2 = x/r
    y2 = y/r
    r = np.sqrt(x**2 + y**2)
    z2 = z/r
    return x2,y2,z2

x2, y2, z2 = [], [], []

for index, row in hits.iterrows():
    x2_, y2_, z2_ = strange(row['x'], row['y'], row['z'])
    x2.append(x2_)
    y2.append(y2_)
    z2.append(z2_)

In [ ]:
plt.scatter(x2, y2, s=1)
plt.show()

In [ ]:
plt.scatter(z2[1::10], x2[1::10], s=1)
plt.show()

In [ ]:
try_clustering(hits, np.transpose([x2,y2,z2]))

In [ ]:
# some figures

# Get every 100th particle
tracks = truth.particle_id.unique()[1::500]

plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
for track in tracks:
    t = truth[truth.particle_id == track]
    ax.plot3D(t.tz, t.tx, t.ty)
ax.set_xlabel('z (mm)')
ax.set_ylabel('x (mm)')
ax.set_zlabel('y (mm)')
# These two added to widen the 3D space
ax.scatter(3000,3000,3000, s=0)
ax.scatter(-3000,-3000,-3000, s=0)
plt.show()